# MAP-Elites track generations

In [1]:
import numpy as np
import requests
import json
import random
from ribs.archives import GridArchive
from ribs.emitters import EmitterBase
from ribs.schedulers import Scheduler
from ribs.visualize import grid_archive_heatmap
import matplotlib.pyplot as plt


from dask.distributed import Client, LocalCluster


import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
# Set up Dask client
cluster = LocalCluster(processes=True, n_workers=12, threads_per_worker=1)
client = Client(cluster)

In [3]:
# Server JS which exposes the trackGenerations and genetic operators 
BASE_URL = 'http://localhost:4242'
POINTS_COUNT = 50
MAX_SELECTED_CELLS = 10
SOLUTION_DIM = POINTS_COUNT * 2 + MAX_SELECTED_CELLS * 2 + 1 
TRACK_SIZE_RANGE = (2, 5)
LENGTH_RANGE = (400, 2000)


# MapElite parameters
ITERATIONS = 100;
ARCHIVE_DIM = 4;
INIT_POPULATION = ARCHIVE_DIM * ARCHIVE_DIM 
MAX_RETRIES = 10 # Maximum number of retries to get a valid solution

# debug flags
DEBUG_CROSSOVER = True
DEBUG_MUTATION = True

### Helper functions

In [4]:
def generate_solution(iteration):
    response = requests.post(f'{BASE_URL}/generate', json={
        "id": iteration + random.random(),
        "mode": "voronoi",
        "trackSize": random.randint(TRACK_SIZE_RANGE[0], TRACK_SIZE_RANGE[1])
    })
    response.raise_for_status()
    return response.json()
    
def solution_to_array(solution):
    array = np.zeros(SOLUTION_DIM)
    # Fill in dataSet
    for i, point in enumerate(solution["dataSet"]):
        array[i*2] = point["x"]
        array[i*2 + 1] = point["y"]
    # Fill in selectedCells
    for i, cell in enumerate(solution["selectedCells"]):
        if i < MAX_SELECTED_CELLS:
            array[POINTS_COUNT*2 + i*2] = cell["x"]
            array[POINTS_COUNT*2 + i*2 + 1] = cell["y"]
    array[-1] = solution["id"]
    return array

def array_to_solution(array):
    dataSet = [{"x": float(array[i]), "y": float(array[i+1])} for i in range(0, POINTS_COUNT*2, 2)]
    selectedCells = [{"x": float(array[i]), "y": float(array[i+1])} 
                     for i in range(POINTS_COUNT*2, SOLUTION_DIM-1, 2) 
                     if array[i] != 0 or array[i+1] != 0]
    
    return {
        "id": float(array[-1]),
        "mode": "voronoi",
        "dataSet": dataSet,
        "selectedCells": selectedCells
    }

def get_fractional_part(id_value):
    return id_value - int(id_value)

def evaluate_solution(solution):
    try:
        response = requests.post(f'{BASE_URL}/evaluate', json=solution)
        response.raise_for_status()
        result = response.json()
        fitness = result['fitness']
        objective = -(fitness['deltaX'] + fitness['deltaY'])
        measures = [len(solution['selectedCells']), fitness['length']]
        return objective, measures
    except requests.RequestException as e:
        print(f"Error in evaluate_solution: {e}")
        return -np.inf, [TRACK_SIZE_RANGE[0], LENGTH_RANGE[0]]

## Genetic operators

In [5]:
class CustomEmitter(EmitterBase):
    def __init__(self, archive, solution_dim, batch_size=ARCHIVE_DIM, bounds=None):
        super().__init__(archive, solution_dim=solution_dim, bounds=bounds)
        self.batch_size = batch_size
        self.iteration = 0

    def ask(self):
        self.iteration += 1
        if self.iteration <= INIT_POPULATION: 
            return np.array([solution_to_array(generate_solution(self.iteration-1)) for _ in range(self.batch_size)])
        elif np.random.random() < 0.5:  # Mutation
            return self.mutate_solutions()
        else:  # Crossover
            return self.crossover_solutions()

    def generate_initial_solutions(self):
        return np.array([solution_to_array(generate_solution(self.iteration-1)) for _ in range(self.batch_size)])


    def mutate_solutions(self):
        parents_data = self.archive.sample_elites(self.batch_size)
        mutated = []

        for i in range(self.batch_size):
            solution = parents_data['solution'][i]
            parent_dict = array_to_solution(solution)

            response = requests.post(f'{BASE_URL}/mutate', json={
                "individual": parent_dict,
                "intensityMutation": 10
            })
            response.raise_for_status()
            mutated_dict = response.json()['mutated']
            
            # Keep the same ID for mutations, just update the iteration part
            parent_frac = get_fractional_part(parent_dict['id'])
            mutated_dict['id'] = self.iteration - 1 + parent_frac
            
            mutated.append(solution_to_array(mutated_dict))

        return np.array(mutated)

    def crossover_solutions(self):
        offspring = []
        for i in range(self.batch_size // 2):
            parents = self.archive.sample_elites(2)
            
            parent1 = array_to_solution(parents['solution'][0])
            parent2 = array_to_solution(parents['solution'][1])
            
            response = requests.post(f'{BASE_URL}/crossover', json={
                "mode": "voronoi",
                "parent1": parent1,
                "parent2": parent2
            })
            response.raise_for_status()
            children = response.json()['offspring']
            # Create a new ID based on parents' IDs
            parent1_frac = get_fractional_part(parent1['id'])
            parent2_frac = get_fractional_part(parent2['id'])
            child_frac = (parent1_frac + parent2_frac) % 1  # Ensure it's between 0 and 1
            new_id = self.iteration - 1 + child_frac
            
            child = {
                "id": new_id,
                "mode": "voronoi",
                "trackSize": len(children['sel']),
                "dataSet": children['ds'],
                "selectedCells": children['sel']
            }
            
            offspring.append(solution_to_array(child))
            if DEBUG_CROSSOVER:
                logger.info(f"Crossover {i+1}, Child: Parent1 ID {parent1['id']}, Parent2 ID {parent2['id']}, Child ID {new_id}")
        
        return np.array(offspring)

## Illuminating search spaces by mapping elites


In [6]:
import sys
import numpy as np
import logging

def run_map_elites(iterations):
    archive = GridArchive(solution_dim=SOLUTION_DIM,
                          dims=[ARCHIVE_DIM, ARCHIVE_DIM],
                          ranges=[TRACK_SIZE_RANGE, LENGTH_RANGE])

    emitter = CustomEmitter(archive,
                            solution_dim=SOLUTION_DIM,
                            batch_size=INIT_POPULATION,
                            bounds=[(0, 600)] * (SOLUTION_DIM - 1) + [(0, float('inf'))])

    scheduler = Scheduler(archive, [emitter])

    for itr in range(iterations):
        try:
            solution_batch = scheduler.ask()
            
            # Use Dask for parallel evaluation
            futures = client.map(evaluate_solution, [array_to_solution(sol) for sol in solution_batch])
            results = client.gather(futures)

            objectives = []
            measures_list = []

            for result in results:
                objective, measures = result
                if np.isfinite(objective):
                    objectives.append(objective)
                    measures_list.append(measures)
                else:
                    logging.warning(f"Non-finite objective value: {objective}. Using placeholder.")
                    objectives.append(-sys.float_info.max)  # Use max negative float as placeholder
                    measures_list.append(measures)  # Keep the original measures

            if objectives and measures_list:
                print(f"Before tell: Archive size = {len(archive)}")
                scheduler.tell(objectives, measures_list)
                print(f"After tell: Archive size = {len(archive)}")
                print(f"Objectives: {objectives}")
                print(f"Measures: {measures_list}")
            else:
                logging.warning(f"Iteration {itr + 1}: No valid solutions to add to archive")

            logging.info(f"> {itr + 1} iterations completed")
            logging.info(f"  - Archive size: {len(archive)}")
            logging.info(f"  - Archive coverage: {archive.stats.coverage}")
            
        except Exception as e:
            logging.error(f"Error in iteration {itr}: {str(e)}")
            raise

    return archive

In [7]:
final_archive = run_map_elites(ITERATIONS)

Before tell: Archive size = 0


INFO:root:> 1 iterations completed
INFO:root:  - Archive size: 9
INFO:root:  - Archive coverage: 0.5625


After tell: Archive size = 9
Objectives: [-0.334949, -3.4755320000000003, -2.101273, -4.228394, -4.294345, -0.30462599999999995, -1.2618109999999998, 4.181503, 3.5234829999999997, 2.0607, -0.42637700000000006, -1.338981, -0.578705, -2.118952, -4.055977, -1.922275]
Measures: [[4, 618.066162], [3, 653.303528], [5, 1387.949829], [3, 1164.506348], [2, 663.530029], [4, 872.62323], [2, 609.741089], [5, 1076.065796], [4, 891.365662], [2, 924.903442], [2, 543.066589], [5, 1129.28186], [5, 637.63385], [4, 980.868164], [3, 656.39502], [2, 486.39679]]


INFO:root:> 2 iterations completed
INFO:root:  - Archive size: 9
INFO:root:  - Archive coverage: 0.5625


Before tell: Archive size = 9
After tell: Archive size = 9
Objectives: [-1.187683, -1.850738, -3.696496, -2.319603, -6.476722, -1.695613, 1.287445, 4.395652999999999, 1.00782, -0.009940999999999978, -2.736267, -6.113602, -1.317375, -2.879578, -2.078629, 1.977447]
Measures: [[5, 1047.397705], [3, 667.458435], [3, 850.025085], [4, 383.894348], [5, 1149.490723], [2, 766.079529], [2, 825.414368], [3, 805.028381], [3, 801.964172], [4, 1172.15564], [3, 417.582153], [5, 879.40686], [3, 501.734528], [2, 714.031494], [5, 1473.855835], [3, 720.641052]]


...

INFO:root:> 46 iterations completed
INFO:root:  - Archive size: 15
INFO:root:  - Archive coverage: 0.9375


Before tell: Archive size = 15
After tell: Archive size = 15
Objectives: [-2.5904849999999997, -3.44381, 0.7582549999999999, -1.7976931348623157e+308, -4.0815660000000005, -1.922638, -0.02525299999999997, -3.62706]
Measures: [[3, 957.614014], [3, 1204.640869], [5, 1329.416016], [2, 400], [3, 1687.402466], [3, 406.064697], [5, 1590.342529], [4, 811.120361]]


INFO:root:> 47 iterations completed
INFO:root:  - Archive size: 15
INFO:root:  - Archive coverage: 0.9375
INFO:__main__:Crossover 1, Child: Parent1 ID 33.8414254310058, Parent2 ID 26.516291365008982, Child ID 47.357716796014785
INFO:__main__:Crossover 2, Child: Parent1 ID 1.841925783759605, Parent2 ID 35.651538725729665, Child ID 47.49346450948927
INFO:__main__:Crossover 3, Child: Parent1 ID 39.01273540968326, Parent2 ID 39.01273540968326, Child ID 47.02547081936652
INFO:__main__:Crossover 4, Child: Parent1 ID 26.809076523697478, Parent2 ID 39.01273540968326, Child ID 47.82181193338074
INFO:__main__:Crossover 5, Child: Parent1 ID 36.87852481998897, Parent2 ID 32.75452271485163, Child ID 47.6330475348406
INFO:__main__:Crossover 6, Child: Parent1 ID 26.809076523697478, Parent2 ID 0.3805266407487209, Child ID 47.1896031644462
INFO:__main__:Crossover 7, Child: Parent1 ID 33.8414254310058, Parent2 ID 26.516291365008982, Child ID 47.357716796014785
INFO:__main__:Crossover 8, Child: Parent1 I

Before tell: Archive size = 15
After tell: Archive size = 15
Objectives: [-1.7976931348623157e+308, -0.9013979999999999, -4.116684, -1.7976931348623157e+308, -0.04895099999999997, 0.2919540000000005, -0.733169, -1.7976931348623157e+308, -12.076798, -1.1192250000000001, -1.7976931348623157e+308, -1.226525, -2.6074219999999997, -2.1726150000000004, 0.761837, -1.7976931348623157e+308]
Measures: [[2, 400], [3, 644.72467], [4, 1270.463379], [2, 400], [5, 1703.252563], [4, 1055.734009], [2, 558.17749], [2, 400], [7, 1732.509521], [2, 957.90448], [2, 400], [3, 626.084595], [5, 1104.715576], [5, 1322.563232], [2, 580.062866], [2, 400]]


KeyboardInterrupt: 

## Visualize Results

In [9]:
plt.figure(figsize=(10, 8))
grid_archive_heatmap(final_archive)
plt.title("MAP-Elites Archive")
plt.xlabel("Track Size")
plt.ylabel("Track Length")
plt.show()

NameError: name 'final_archive' is not defined

<Figure size 1000x800 with 0 Axes>